## Weaviate Query Agent with Ollama

This notebook will show you how to define the Weaviate Query Agent as a tool through Ollama.

### Requirements
1. Weaviate Cloud instance (WCD): The Weaviate Query Agent is only accessible through WCD at the moment. You can create a serverless cluster or a free 14-day sandbox [here](https://console.weaviate.cloud/).
1. Install the Ollama with `pip install ollama`. We're using version `0.4.7` at the time of writing this notebook.
1. Install the Weaviate Agents package with `pip install weaviate-agents`
1. You'll need a Weaviate cluster with data. If you don't have one, check out [this notebook](integrations/Weaviate-Import-Example.ipynb) to import the Weaviate Blogs.


### Import libraries and keys

In [ ]:
import weaviate
from weaviate_agents.query import QueryAgent
import os
import json

import ollama
from typing import List, Dict

In [1]:
os.environ["WEAVIATE_URL"] = ""
os.environ["WEAVIATE_API_KEY"] = ""

### Define Query Agent function

In [ ]:
def send_query_agent_request(query: str) -> str:
    """
    Send a query to the database and get the response.

    Args:
        query (str): The question or query to search for in the database. This can be any natural language question related to the content stored in the database.

    Returns:
        str: The response from the database containing relevant information.
    """

    # connect to your Weaviate Cloud instance
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.getenv("WEAVIATE_URL"), 
        auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),
        headers={ # add the API key to the model provider from your Weaviate collection, for example `headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}` 
        }
    )

    # connect the query agent to your Weaviate collection(s)
    query_agent = QueryAgent(
        client=weaviate_client,
        collections=["Blogs"] 
    )
    return query_agent.run(query).final_answer

### Define tool

In [7]:
query_agent_tool_schema = [{
    'type': 'function',
    'function': {
        'name': 'send_query_agent_request',
        'description': 'Send a query to the Weaviate database and get relevant information from blog posts.',
        'parameters': {
            'type': 'object',
            'properties': {
                'query': {
                    'type': 'string',
                    'description': 'The search query to find information in the blog posts database.',
                },
            },
            'required': ['query'],
        },
    },
}]

### Create function calling loop

In [8]:
tool_mapping = {
    "send_query_agent_request": send_query_agent_request
}

def ollama_generation_with_weaviate_agent(user_message: str,
                                          model_name: str = "llama3.1") -> str:
    """
    Generate a response using Ollama with access to the Weaviate Query Agent.
    
    Args:
        user_message (str): The user's query or message
        model_name (str): The name of the Ollama model to use
        
    Returns:
        str: The final response after potential tool execution
    """
    messages = [{
        "role": "user",
        "content": user_message
    }]
    
    # First call to decide if tool should be used
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=query_agent_tool_schema
    )
    
    # Add model response to message history
    messages.append(response["message"])
    
    # Check if the model decided to use a tool
    if response["message"].get("tool_calls"):
        print("Model decided to use tool...")
        
        for tool in response["message"]["tool_calls"]:
            function_name = tool["function"]["name"]
            
            if function_name in tool_mapping:
                # Parse arguments - handle different possible formats
                try:
                    # If arguments is a JSON string
                    if isinstance(tool["function"]["arguments"], str):
                        args = json.loads(tool["function"]["arguments"])
                        query = args.get("query", "")
                    # If arguments is already parsed as a dict
                    elif isinstance(tool["function"]["arguments"], dict):
                        query = tool["function"]["arguments"].get("query", "")
                    else:
                        query = ""
                    
                    print(f"Querying Weaviate with: {query}")
                    
                    # Call the function with the query
                    function_to_call = tool_mapping[function_name]
                    function_response = function_to_call(query)
                    
                    # Make sure response is a string
                    if not isinstance(function_response, str):
                        if isinstance(function_response, dict) and 'final_answer' in function_response:
                            function_response = function_response['final_answer']
                        else:
                            function_response = json.dumps(function_response)
                    
                    # Add the tool response to messages
                    messages.append({
                        "role": "tool",
                        "content": function_response,
                        "name": function_name  # Include the function name
                    })
                    
                except Exception as e:
                    error_message = f"Error executing tool: {str(e)}"
                    print(error_message)
                    messages.append({
                        "role": "tool",
                        "content": error_message,
                        "name": function_name
                    })
        
        # Get final response with tool results
        final_response = ollama.chat(model=model_name, messages=messages)
        return final_response["message"]["content"]
    else:
        # If no tool was used, return the direct response
        return response["message"]["content"]

### Query time

In [9]:
# Example usage
response = ollama_generation_with_weaviate_agent(
    user_message="How do I deploy Weaviate with Docker?",
    model_name="llama3.1"  # or whichever model you have in Ollama
)
print(response)

Model decided to use tool...
Querying Weaviate with: docker deployment
To deploy Weaviate with Docker, you'll need to follow these steps:

1. **Pull the Weaviate Docker image**: You can pull the official Weaviate Docker image from Docker Hub using:
   ```bash
   docker pull weaviate/weaviate:latest
   ```

2. **Run Weaviate in a container**:
   If you're on Linux, use `docker run` to start Weaviate in a new container. For example, to expose the default port (8087) and mount a local directory (`/path/to/your/data`) as a volume inside the container, you can use the following command:

   ```bash
   docker run -p 8080:8080 -v /path/to/your/data:/data weaviate/weaviate:latest --memory 4g --indexing 2 --storage 1 --port 8080
   ```

3. **Verify Weaviate**:
   After running the command, you can verify that Weaviate is up and running by accessing `http://localhost:8080` in your browser. The default admin credentials are username "admin" and password "password".

4. **Stop or remove containers